In [1]:
#Load the 'insurance' dataset
import pandas as pd
df = pd.read_csv('insurance.csv')
df.head()

,age,affordability,have_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [6]:
#split the dataset into training and testing dataset
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df[['age', 'affordability']], df.have_insurance, train_size = 0.7)
x_test.head(), x_train.shape

(    age  affordability
 23   45              1
 6    55              0
 21   26              0
 4    46              1
 0    22              1,
 (19, 2))

In [7]:
#scale the 'age' feature in the dataset
x_train_scaled = x_train.copy()
x_train_scaled['age'] = x_train_scaled['age'] / 100

x_test_scaled = x_test.copy()
x_test_scaled['age'] = x_test_scaled['age'] / 100

x_test_scaled['age'].head()

23    0.45
6     0.55
21    0.26
4     0.46
0     0.22
Name: age, dtype: float64

<h2>Implement the MyNN class which includes 'predict', 'fit' methods</h2>

In [8]:
#Modules gonna be used in 'MyNN' class
import numpy as np

In [9]:
class MyNN:
    #constructor
    def __init__(self) -> None:
        self.w1 = 1
        self.w2 = 1
        self.bias = 0
        self.learning_rate = 0.01

    #sigmoid function
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    #log loss function
    def binary_cross_entropy(self, y_true, y_pred):
        epsilon = 1e-15
        #To replace the '0' in y_pred coz log(0) is undefined or infinity
        y_pred_new = [max(i, epsilon) for i in y_pred]
        #To replace the '1' in y_pred coz log(1) is undefined or infinity
        y_pred_new = [min(i, 1 - epsilon) for i in y_pred_new]
        y_pred_new = np.array(y_pred_new)
        return -(np.mean( y_true * np.log(y_pred_new) + (1 - y_true) * np.log(1 - y_pred_new)))

    #prediction function
    def prediction_function(self, age, affordability):
        weighted_sum = self.w1 * age + self.w2 * affordability + self.bias
        y_predict = self.sigmoid(weighted_sum)
        return y_predict

    #gradient descent function to find w1, w2 and bias
    def gradient_descent(self, age, affordability, y_true, epochs, loss_thresold):        
        n = len(age)
        for i in range(epochs):
            #calculate the loss using log loss function (or) binary cross entropy
            y_predicted = self.prediction_function(age, affordability)
            loss = self.binary_cross_entropy(y_true, y_predicted)

            #caculate the partial derivatives of w1, w2 and bias
            w1d = 1/n * np.dot(np.transpose(age), (y_predicted - y_true))
            w2d = 1/n * np.dot(np.transpose(affordability), (y_predicted - y_true))
            bias_d = np.mean(y_predicted - y_true)

            #calculate the next w1, w2 and bias using 'partial derivatives and slopes'
            self.w1 = self.w1 - (self.learning_rate * w1d)
            self.w2 = self.w2 - (self.learning_rate * w2d)
            self.bias = self.bias - (self.learning_rate * bias_d)

            print(f'epoch: {i+1}, w1: {self.w1}, w2: {self.w2}, bias: {self.bias}, loss: {loss}')

            if( loss <= loss_thresold):
                print(f'epoch: {i+1}, w1: {self.w1}, w2: {self.w2}, bias: {self.bias}, loss: {loss}')                
                break

        #return tuple of values include w1, w2, bias
        return self.w1, self.w2, self.bias

    #'fit' method to train the model
    def fit(self, x, y, epochs):
        w1, w2, bias = self.gradient_descent(x['age'], x['affordability'], y, epochs, 0.4613)
        print("\n\n")
        print(f'Intercepts: {[w1, w2]}, Bias: {bias}')
        print("\n Model trained")

    #'predict' method to predict the output
    def predict(self, x):
        result = self.prediction_function(x['age'], x['affordability'])
        print(result)

In [10]:
#Train my own model
model = MyNN()
model.fit(x_train_scaled, y_train, 1000)

epoch: 1, w1: 0.9996059035301984, w2: 0.9987891474101672, bias: -0.0022394496027272064, loss: 0.6976893967006198
epoch: 2, w1: 0.9992141332467362, w2: 0.9975824335376616, bias: -0.004473253260908113, loss: 0.6970266639527967
epoch: 3, w1: 0.9988246866275369, w2: 0.9963798551713864, bias: -0.006701417640629834, loss: 0.6963676364426499
epoch: 4, w1: 0.9984375611198492, w2: 0.9951814090345502, bias: -0.008923949478020325, loss: 0.6957122993435814
epoch: 5, w1: 0.9980527541404473, w2: 0.993987091785031, bias: -0.011140855578755954, loss: 0.6950606378139965
epoch: 6, w1: 0.9976702630758312, w2: 0.9927969000157418, bias: -0.013352142817566678, loss: 0.6944126369981746
epoch: 7, w1: 0.9972900852824289, w2: 0.9916108302550007, bias: -0.015557818137738877, loss: 0.6937682820271378
epoch: 8, w1: 0.9969122180867996, w2: 0.9904288789669029, bias: -0.017757888550615903, loss: 0.6931275580195134
epoch: 9, w1: 0.9965366587858375, w2: 0.9892510425516956, bias: -0.019952361135096446, loss: 0.692490450

In [12]:
model.predict(x_test_scaled), y_test

23    0.624496
6     0.450246
21    0.367797
4     0.627258
0     0.559081
25    0.649039
16    0.365059
12    0.370543
10    0.547422
dtype: float64


(None,
 23    1
 6     0
 21    0
 4     1
 0     0
 25    1
 16    1
 12    0
 10    0
 Name: have_insurance, dtype: int64)